In [1]:
cd ..

C:\Users\wpgur\OneDrive\프로그래밍\Back-End\tdatlib


In [2]:
from tdatlib import macro, market, index, stock
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important;}</style>"))

# Initialize

In [3]:
macro = macro()
market = market.KR()
index = index()
index.data.period = macro.data.period = 20

tickers = pd.DataFrame(
    data=[
        dict(ticker="5046",   종류="지수", 이름="KRX은행"),
        dict(ticker="091170", 종류="ETF", 이름="KODEX은행"),
        dict(ticker="091220", 종류="ETF", 이름="TIGER은행"),
        dict(ticker="105560", 종류="주식", 이름="KB금융"),
        dict(ticker="086790", 종류="주식", 이름="하나금융지주"),
        dict(ticker="139130", 종류="주식", 이름="DGB금융지주"),
        dict(ticker="055550", 종류="주식", 이름="신한지주"),
        dict(ticker="024110", 종류="주식", 이름="기업은행"),
        dict(ticker="138930", 종류="주식", 이름="BNK금융지주"),
        dict(ticker="316140", 종류="주식", 이름="우리금융지주"),
        dict(ticker="038540", 종류="주식", 이름="상상인"),
    ]
).set_index(keys='ticker').join(other=market.icm)

stock_objs = {ticker: stock.kr(ticker=ticker) for ticker in tickers[tickers.종류 == '주식'].index}

In [4]:
# 코스피 리세션
def add_recession(fig:go.Figure):
    start = min([d['x'][0] for d in fig['data']])
    for d in macro.data.KR_recession:
        x0, x1 = d['from'], d['to']
        if x1 < start:
            continue
        if x0 < start:
            x0 = start
        fig.add_vrect(x0=x0, x1=x1, fillcolor='grey', opacity=0.2, line_width=0)
    return fig

# 레이아웃
layout = dict(
    plot_bgcolor='white',
    height=750,
    legend=dict(
        orientation="h",
        xanchor="right", yanchor="bottom",
        x=0.96, y=1,
    ),
)

# Data Collection

## KRX은행지수

In [5]:
# KRX 은행 지수
krx_bank = index.data.bank
krx_bank_trace = go.Scatter(
    x=krx_bank.index,
    y=krx_bank.종가,
    name='KRX은행지수',
    showlegend=True,
    line=dict(
        color='black',
        width=1
    ),
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}'
)

## 수신/대출 금리 정보

In [6]:
# 수신금리
depo_by_new = macro.data.ecos('121Y002') # 신규 취급액 기준
depo_by_left = macro.data.ecos('121Y013') # 잔액 기준

# 대출금리
loan_by_new = macro.data.ecos('121Y006') # 신규 취급액 기준
loan_by_left = macro.data.ecos('121Y015') # 잔액 기준

col_table = pd.concat(
    objs={
        '수신/신규취급':pd.Series(depo_by_new.columns),
        '수신/잔액':pd.Series(depo_by_left.columns),
        '대출/신규취급':pd.Series(loan_by_new.columns),
        '대출/잔액':pd.Series(loan_by_left.columns)
    },
    axis=1
)
col_table

0                           양도성예금증서
1                              상호부금
2                환매조건부채권매도(91-180일)
3                       정기예금(5년 이상)
4                              정기적금
5                        상호부금(3-4년)
6                              주택부금
7                           시장형금융상품
8                              표지어음
9                     표지어음(91-120일)
10                  정기예금(6개월 미만) 2)
11                 저축성수신(금융채 제외) 1)
12                       순수저축성예금 1)
13                    정기예금(2~3년 미만)
14                        환매조건부채권매도
15                     양도성예금증서(91일)
16                             정기예금
17               정기예금(6개월~1년 미만) 2)
18                    정기예금(3~4년 미만)
19                    정기예금(4~5년 미만)
20                       정기적금(3-4년)
21                 정기예금(1~2년 미만) 2)
22                           금융채 3)
23                            저축성수신
24                    저축성수신(금융채 제외)
25           저축성수신(요구불예금 및 수시입출식제외)
26                            순수저축성
34                          

,수신/신규취급,수신/잔액,대출/신규취급,대출/잔액
0,환매조건부채권매도(91-180일),총수신(요구불예금_및_수시입출식_저축성예금_포함),대출평균_1),총대출_1)
1,정기예금(5년_이상),수시입출식_저축성예금,기업대출,총대출(당좌대출_제외)
2,상호부금(3-4년),저축성수신(요구불예금_및_수시입출식_저축성예금_제외),가계대출,기업대출_2)
3,주택부금,저축성수신(금융채_제외),소액대출_(500만원_이하),대기업대출
4,표지어음(91-120일),순수저축성예금,주택담보대출,중소기업대출
5,정기예금(6개월_미만)_2),정기예금,예·적금담보대출,운전자금대출
6,저축성수신(금융채_제외)_1),정기적금,보증대출,시설자금대출
7,순수저축성예금_1),상호부금,일반신용대출_2),가계대출_2)
8,정기예금(2-3년_미만),주택부금,집단대출_2),소액대출_(500만원_이하)
9,양도성예금증서(91일),시장형금융상품,공공및기타부문대출,주택담보대출


In [7]:
# col_depo = '총수신(요구불예금_및_수시입출식_저축성예금_포함)'
# col_depo = '저축성수신(금융채_제외)_1)'
col_depo = '저축성수신(요구불예금_및_수시입출식제외)'
# col_loan = '대출평균_1)'
# col_loan = '총대출(당좌대출제외)'
col_loan = '기업대출'

depo = depo_by_new[col_depo].astype(float).copy()
loan = loan_by_new[col_loan].astype(float).copy()
depo_loan_ratio = loan / depo
depo_loan_diff = loan - depo

depo_trace = go.Scatter(
    name=col_depo,
    x=depo.index, y=depo,
    showlegend=True,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

loan_trace = go.Scatter(
    name=col_loan,
    x=loan.index, y=loan,
    showlegend=True,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

l_d_diff = loan - depo
l_d_diff_trace = go.Scatter(
    name='대출/수신금리-차',
    x=l_d_diff.index, y=l_d_diff,
    showlegend=True,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

l_d_ratio = loan/depo
l_d_ratio_trace = go.Scatter(
    name='대출/수신금리-비',
    x=l_d_ratio.index, y=l_d_ratio,
    showlegend=True,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

## 기준 금리

In [8]:
# 한국은행 기준금리
kr_std = macro.data.KR_IR
kr_std_trace = go.Scatter(
    x=kr_std.index,
    y=kr_std,
    name='한국기준금리',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

# 미국 기준금리
us_std = macro.data.US_IR
us_std_trace = go.Scatter(
    x=us_std.index,
    y=us_std,
    name='미국기준금리',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

0                 한국은행 기준금리
7580                정부대출금금리
15127              총액한도대출금리
19245         결제자금지원한도 대출금리
19326       무역금융지원 프로그램대출금리
22755       신용대출지원 프로그램대출금리
23566     영세자영업자지원 프로그램대출금리
26995    신성장·일자리지원 프로그램대출금리
30424       설비투자지원 프로그램대출금리
33345     지방중소기업지원 프로그램대출금리
36774             자금조정 대출금리
40663             자금조정 예금금리
Name: ITEM_NAME1, dtype: object
한국은행 기준금리 OK
정부대출금금리 OK
총액한도대출금리 OK
결제자금지원한도 대출금리 OK
무역금융지원 프로그램대출금리 OK
신용대출지원 프로그램대출금리 OK
영세자영업자지원 프로그램대출금리 OK
신성장·일자리지원 프로그램대출금리 OK
설비투자지원 프로그램대출금리 OK
지방중소기업지원 프로그램대출금리 OK
자금조정 대출금리 OK
자금조정 예금금리 OK


## 2년물 금리

In [9]:
# 한국채 2년물 금리
kr_2yt = macro.data.KR_2Y_TY
# kr_2yt = macro.data.ecos('817Y002')['통안증권(2년)']
kr_2yt_trace = go.Scatter(
    x=kr_2yt.index,
    y=kr_2yt,
    name='한국국고채2년물',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

# 미국채 2년물 금리
us_2yt = macro.data.US_2Y_TY
us_2yt_trace = go.Scatter(
    x=us_2yt.index,
    y=us_2yt,
    name='미국국고채2년물',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

0                 국고채(3년)
1         콜금리(1일, 중개회사거래)
2           콜금리(1일, 전체거래)
3                통안증권(2년)
4                 국고채(1년)
5           회사채(3년, BBB-)
6            회사채(3년, AA-)
7                국고채(10년)
8                CMA(수시형)
9                 CP(91일)
10                CD(91일)
11                MMF(7일)
12               통안증권(1년)
13                국고채(5년)
2235     회사채(3년, AA-, 민평)
6373              산금채(1년)
6377         국민주택채권1종(5년)
9821        KORIBOR(12개월)
14409        KORIBOR(3개월)
16768            국고채(20년)
16850        KORIBOR(6개월)
20231           통안증권(91일)
53015            국고채(30년)
76178            국고채(50년)
Name: ITEM_NAME1, dtype: object
국고채(3년) OK
콜금리(1일, 중개회사거래) OK
콜금리(1일, 전체거래) OK
통안증권(2년) OK
국고채(1년) OK
회사채(3년, BBB-) OK
회사채(3년, AA-) OK
국고채(10년) OK
CMA(수시형) OK
CP(91일) OK
CD(91일) OK
MMF(7일) OK
통안증권(1년) OK
국고채(5년) OK
회사채(3년, AA-, 민평) OK
산금채(1년) OK
국민주택채권1종(5년) OK
KORIBOR(12개월) OK
KORIBOR(3개월) OK
국고채(20년) OK
KORIBOR(6개월) OK
통안증권(91일) OK
국고채(30년) OK
국고채(50년) OK


KeyError: '국고채(2년)'

## 10년물 금리

In [ ]:
# 한국채 10년물 금리
kr_10yt = macro.data.KR_10Y_TY
kr_10yt_trace = go.Scatter(
    x=kr_10yt.index,
    y=kr_10yt,
    name='한국국고채10년물',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

# 미국채 10년물 금리
us_10yt = macro.data.US_10Y_TY
us_10yt_trace = go.Scatter(
    x=us_10yt.index,
    y=us_10yt,
    name='미국국고채10년물',
    showlegend=True,
    line_width=1.2,
    xhoverformat='%Y/%m/%d',
    hovertemplate='%{x}<br>%{y:.2f}%'
)

## 원-달러 환율

In [ ]:
# 원-달러 환율
exchange = macro.data.KRW_USD_exchange
exchange_trace = go.Candlestick(
    name='원-달러 환율',
    x=exchange.index,
    open=exchange.시가,
    high=exchange.고가,
    low=exchange.저가,
    close=exchange.종가,
    visible=True,
    showlegend=True,
    increasing_line=dict(color='red'),
    decreasing_line=dict(color='royalblue'),
    xhoverformat='%Y/%m/%d',
    yhoverformat='.2f',
)

# BM & Products

- 은행 사업 구조 및 최대 매출처 확인
- 주요 매출처
  * 유가 증권 평가 및 처분 이익<br>
    : 1년 또는 1영업주기 내 현금화할 의도가 있는 증권을 유가 증권이라고 한다.<br>
    : 증권은 CD, CP 등 단기 증서 및 보통주/우선주의 주식과 사채, 어음 등 채무 증권을 말한다.<br>
    : 이러한 유가 증권을 취득 가액보다 높은 이익으로 처분할 때 발생하는 이익 및 보유 증권의 평가액을 매출에 반영한다.<br>
  * 신탁 업무 운용 수익
    : 운용

## 대표 시중은행 매출 구조

In [ ]:
target = tickers.sort_values(by='시가총액', ascending=False)[:4]
fig = make_subplots(
    rows=2, cols=2,
    vertical_spacing=0.12, horizontal_spacing=0.1,
    subplot_titles=tuple(target.이름.tolist()),
    specs=[
        [{"type": "domain"}, {"type": "domain"}],
        [{"type": "domain"}, {"type": "domain"}]
    ]
)

for n, ticker in enumerate(target.index):
    obj = stock_objs[ticker]
    products = obj.fnguide.Products
    fig.add_trace(
        trace = go.Pie(
            name='Product',
            labels=products.index,
            values=products,
            visible=True,
            showlegend=True,
            automargin=True,
            textfont=dict(color='white'),
            textinfo='label+percent',
            insidetextorientation='radial',
            hoverinfo='label+percent',
        ), row = n // 2 + 1, col = n % 2 + 1
    )

fig.update_layout(layout)
fig.update_layout(    
    title=f'<b>은행주</b> 주요 매출',
    legend=dict(y=1.08)
)

for n, annotation in enumerate(fig['layout']['annotations']):
    annotation['x'] = 0 + 0.55 * (n % 2)
    annotation['xanchor'] = 'center'
    annotation['xref'] = 'paper'
fig.show()

## 매출 비중

In [ ]:
p = pd.concat(objs=[stock_objs[ticker].fnguide.Products for n, ticker in enumerate(target.index)], axis=1).T.sum()
portion = round(100 * p / p.sum(), 2).sort_values()

fig = go.Figure()
fig.add_trace(
    trace=go.Bar(
        x=portion, y=portion.index, orientation='h',
        texttemplate='%{x:.2f}%', textposition='inside', hoverinfo='skip'
    )
)
fig.update_layout(
    title='시중은행 주요 매출처 비중', plot_bgcolor='white', height=500,
    xaxis=dict(title='비중[%]', showgrid=True, gridcolor='lightgrey', showline=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
)

# Macro-Relation

## KRX은행 vs 금리

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
# fig.add_trace(trace=krx_bank_trace, secondary_y=False)
# fig.add_trace(trace=depo_trace, secondary_y=True)
# fig.add_trace(trace=loan_trace, secondary_y=True)
# fig.add_trace(trace=l_d_diff_trace, secondary_y=True)
# fig.add_trace(trace=l_d_ratio_trace, secondary_y=True)
fig.add_trace(trace=kr_std_trace, secondary_y=True)
fig.add_trace(trace=us_std_trace, secondary_y=True)
fig.add_trace(trace=kr_2yt_trace, secondary_y=True)
fig.add_trace(trace=us_2yt_trace, secondary_y=True)
# fig.add_trace(trace=us_10yt_trace, secondary_y=True)

add_recession(fig)
fig.update_layout(layout)
fig.update_layout(
    title=f'<b>KRX은행지수</b>와 금리',
    xaxis=dict(
        title='날짜',
        tickformat='%Y/%m/%d',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        title='지수',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis2=dict(
        title='이율[%]',
        showticklabels=True
    )
)

## 원-달러 환율과 KRX은행지수

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(trace=krx_bank_trace, secondary_y=False)
fig.add_trace(trace=exchange_trace, secondary_y=True)
add_recession(fig)
fig.update_layout(layout)
fig.update_layout(
    title=f'<b>KRX은행지수</b>와 원-달러 환율',
    xaxis=dict(
        title='날짜',
        tickformat='%Y/%m/%d',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey',
    ),
    yaxis=dict(
        title='지수',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis2=dict(
        title='KRW',
        showticklabels=True
    ),
    xaxis_rangeslider=dict(visible=False)
)

# Foreigners

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    vertical_spacing=0.12, horizontal_spacing=0.1,
    subplot_titles=tuple(target.이름.tolist()),
    specs=[
        [{"type": "xy", "secondary_y": True}, {"type": "xy", "secondary_y": True}],
        [{"type": "xy", "secondary_y": True}, {"type": "xy", "secondary_y": True}]
    ]
)

for n, ticker in enumerate(target.index):
    obj = stock_objs[ticker]
    foreigner = obj.fnguide.Foreigner['3Y'].dropna()
    fig.add_trace(
        trace = go.Scatter(
            name=ticker,
            x=foreigner.index, y=foreigner['외국인보유비중'],
            showlegend=False,
            xhoverformat='%Y/%m/%d', 
            hovertemplate='%{x}<br>%{y:.2f}%<extra>보유비중</extra>'
        ), row = n // 2 + 1, col = n % 2 + 1, secondary_y=False
    )
    fig.add_trace(
        trace = go.Scatter(
            name=ticker,
            x=foreigner.index, y=foreigner['종가'],
            line=dict(color='#000000', dash='dot', width=1),
            showlegend=False,
            xhoverformat='%Y/%m/%d', 
            hovertemplate='%{x}<br>%{y:,d}원<extra>종가</extra>'
        ), row = n // 2 + 1, col = n % 2 + 1, secondary_y=True
    )
    

add_recession(fig)
fig.update_layout(layout)
fig.update_layout(    
    title=f'<b>은행주</b> 외국인 보유비중',
    xaxis=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    xaxis2=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    xaxis3=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    xaxis4=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    yaxis=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis2=dict(title='[KRW]', showgrid=False),
    yaxis3=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis4=dict(title='[KRW]', showgrid=False),
    yaxis5=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis6=dict(title='[KRW]', showgrid=False),
    yaxis7=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis8=dict(title='[KRW]', showgrid=False),
)

for n, annotation in enumerate(fig['layout']['annotations']):
    annotation['x'] = 0 + 0.55 * (n % 2)
    annotation['xanchor'] = 'center'
    annotation['xref'] = 'paper'
fig.show()

* 습작 노트

In [ ]:
# 금리 정보
kr_interests = macro.data.ecos('817Y002')
print(f"한국은행 금리정보:\n{kr_interests.columns.to_list()}\n")

In [ ]:
deposit = kr_interests['자금조정 예금금리'].dropna().astype(float)
# loan = kr_interests['자금조정 대출금리'].dropna().astype(float)
loan = kr_interests['정부대출금금리'].dropna().astype(float)
margin = loan - deposit

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=deposit.index,
        y=deposit,
        name='예금금리',
    )
)
fig.add_trace(
    go.Scatter(
        x=loan.index,
        y=loan,
        name='대출금리'
    )
)
fig.add_trace(
    go.Scatter(
        x=margin.index,
        y=margin,
        name='예대마진'
    )
)
